In [2]:
import cohere
co = cohere.Client('')
response = co.chat(
  chat_history=[
    {"role": "USER", "message": "Who discovered gravity?"},
    {"role": "CHATBOT", "message": "The man who is widely credited with discovering gravity is Sir Isaac Newton"}
  ],
  message="What year was he born?",
    model = "command-r",
  # perform web search before answering the question. You can also use your own custom connector.
  connectors=[{"id": "web-search"}]
)
print(response)

cohere.Chat {
	id: 2333208d-90d8-40a1-9662-b0cf94e1fa8f
	response_id: 2333208d-90d8-40a1-9662-b0cf94e1fa8f
	generation_id: 7dd7d8f3-c720-4546-b847-b6ac6ac9e8ff
	message: What year was he born?
	text: Isaac Newton was born on 4 January 1643 (according to the Gregorian calendar in use today). However, if you were using the Julian calendar, which was in use in England at the time of Newton's birth, his birth date would be recorded as 25 December 1642.
	conversation_id: None
	prompt: None
	chat_history: [{'message': 'Who discovered gravity?', 'role': 'USER'}, {'message': 'The man who is widely credited with discovering gravity is Sir Isaac Newton', 'role': 'CHATBOT'}, {'message': 'What year was he born?', 'response_id': '2333208d-90d8-40a1-9662-b0cf94e1fa8f', 'generation_id': 'd6b30eff-15db-4c90-817a-ae12e9faa7a7', 'role': 'USER'}, {'message': "Isaac Newton was born on 4 January 1643 (according to the Gregorian calendar in use today). However, if you were using the Julian calendar, which w

In [10]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings

In [11]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatCohere(cohere_api_key="",
                               model="command-r")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Found model file at  /Users/pradeep.borado/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


objc[58690]: Class GGMLMetalClass is implemented in both /Users/pradeep.borado/.conda/envs/cv/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x13becc208) and /Users/pradeep.borado/.conda/envs/cv/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x13c240208). One of the two will be used. Which one is undefined.


In [12]:
rag_chain.invoke("What is Task Decomposition?")



"Task decomposition is a method of breaking down complex tasks into simpler, more manageable steps. It is a thought process that can be executed by LLMs, with or without human input, to aid an agent's planning. This technique is employed when a task is too complicated to be solved without first being dissected into its constituent parts."

In [20]:
from langchain.retrievers import CohereRagRetriever
from langchain.retrievers import WikipediaRetriever
from langchain_community.chat_models import ChatCohere


# Load the cohere chat model
cohere_chat_model = ChatCohere(cohere_api_key="",
                               model="command-r")
rag = CohereRagRetriever(llm=cohere_chat_model, connectors=[])


In [21]:
user_query = 'What is Task Decomposition?'
docs = retriever.get_relevant_documents(user_query)

docs = rag.get_relevant_documents(user_query ,source_documents=docs)


In [22]:

# Print the final generation 
answer = docs[-1].page_content
print(answer)


Task decomposition is the process of breaking down a complex task into several simpler steps. This is done to make it easier for an agent to plan ahead. There are three ways to decompose a task: by using simple prompts, task-specific instructions or human inputs.
LLMs also struggle with task decomposition, especially when it comes to planning over a long history and exploring solutions. They have a hard time adjusting plans when faced with errors and are thus less robust than humans in this regard.


In [23]:
# tool descriptions that the model has access to
# note: Cohere always adds a "directly_answer" tool under the hood, so that the model can decide to not leverage any tool, if they're not needed.
tools = [
   {
       "name": "query_daily_sales_report",
       "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
       "parameter_definitions": {
           "day": {
               "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
               "type": "str",
               "required": True
           }
       }
   },
   {
       "name": "query_product_catalog",
       "description": "Connects to a a product catalog with information about all the products being sold, including categories, prices, and stock levels.",
       "parameter_definitions": {
           "category": {
               "description": "Retrieves product information data for all products in this category.",
               "type": "str",
               "required": True
           }
       }
   }
]


# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.


## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""


# user request
message = "Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?"


response = co.chat(
   message=message,
   tools=tools,
   preamble=preamble,
   model="command-r"
)

In [24]:
response

cohere.Chat {
	id: 26a7f184-20f1-4875-8506-f8d3f0fc7286
	response_id: 26a7f184-20f1-4875-8506-f8d3f0fc7286
	generation_id: 31b8c6b2-5966-4dae-9413-0749cf30b510
	message: Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?
	text: 
	conversation_id: None
	prompt: None
	chat_history: None
	preamble: None
	client: <cohere.client.Client object at 0x10f421f10>
	token_count: None
	meta: {'api_version': {'version': '1'}, 'billed_units': {'output_tokens': 33}}
	is_search_required: None
	citations: None
	documents: None
	search_results: None
	search_queries: None
	tool_calls: [cohere.ToolCall {
	name: query_daily_sales_report
	parameters: {'day': '2023-09-29'}
	generation_id: 31b8c6b2-5966-4dae-9413-0749cf30b510
}, cohere.ToolCall {
	name: query_product_catalog
	parameters: {'category': 'Electronics'}
	generation_id: 31b8c6b2-5966-4dae-9413-0749cf30b510
}]
	finish_reaso